# TECH CHALLENGE FIAP - 2024

Imagine agora, que você vai atuar como Expert em Data Analytics em uma empresa que exporta vinhos do Brasil para o mundo todo.

Sua área é recém-criada dentro da empresa, e você será responsável pelos relatórios iniciais a serem apresentados em uma reunião de investidores e acionistas, explicando a quantidade de vinhos exportados e os fatores externos que podem vir a surgir e que interferem nas análises:

1. Dados climáticos
2. Dados demográficos
3. Dados econômicos
4. Dados de avaliação de vinhos

O Head de Dados pediu para que você construísse uma tabela contendo as seguintes informações:

1. País de origem
2. País de destino
3. Quantidade (L)
4. Valor em US$

Os dados que lhe forneceram são de uma vinícola parceira, e podem ser encontrados <a href="http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_01" target="_blank">aqui</a>.

Agora, você precisa construir um relatório com as informações solicitadas.

Seu objetivo é dizer o montante de venda de exportação nos últimos 15 anos, separando a análise por país e trazendo quais as prospecções futuras e possíveis ações para uma melhoria nas exportações. Construa gráficos atraentes e que passem a ideia central para que os acionistas e investidores possam seguir em frente com suas ações.

## Análise Explotatória de Dados

In [63]:
import pandas as pd

In [64]:
export_data = pd.read_csv("../data/ExpVinho.csv", sep=";")

In [65]:
export_data.shape

(128, 108)

In [66]:
export_data.head(-5)

,Id,País,1970,1970.1,1971,1971.1,1972,1972.1,1973,1973.1,...,2018,2018.1,2019,2019.1,2020,2020.1,2021,2021.1,2022,2022.1
0,1,Afeganistão,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,11,46,0,0
1,2,África do Sul,0,0,0,0,0,0,0,0,...,0,0,26,95,4,21,0,0,0,0
2,3,"Alemanha, República Democrática",0,0,0,0,4168,2630,12000,8250,...,10794,45382,3660,25467,6261,32605,2698,6741,7630,45367
3,4,Angola,0,0,0,0,0,0,0,0,...,477,709,345,1065,0,0,0,0,4068,4761
4,5,Anguilla,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,119,"Tcheca, República",0,0,0,0,0,0,0,0,...,2712,20980,2115,16391,563,4805,456,5988,1305,9997
119,120,Togo,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1890,2012,17317,25608
120,121,Trinidade Tobago,0,0,0,0,0,0,336,300,...,0,0,0,0,0,0,0,0,0,0
121,122,Tunísia,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Transformação dos dados

In [67]:
export_data.columns[-32:]

Index(['2007', '2007.1', '2008', '2008.1', '2009', '2009.1', '2010', '2010.1',
       '2011', '2011.1', '2012', '2012.1', '2013', '2013.1', '2014', '2014.1',
       '2015', '2015.1', '2016', '2016.1', '2017', '2017.1', '2018', '2018.1',
       '2019', '2019.1', '2020', '2020.1', '2021', '2021.1', '2022', '2022.1'],
      dtype='object')

In [68]:
# Selecionando as colunas de interesse para os dados de exportação de vinhos

total_columns = len(export_data.columns)
selected_export_data = export_data.iloc[
    :, [0, 1] + list(range(total_columns - 32, total_columns))
]
selected_export_data.head()

,Id,País,2007,2007.1,2008,2008.1,2009,2009.1,2010,2010.1,...,2018,2018.1,2019,2019.1,2020,2020.1,2021,2021.1,2022,2022.1
0,1,Afeganistão,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,11,46,0,0
1,2,África do Sul,0,0,0,0,0,0,0,0,...,0,0,26,95,4,21,0,0,0,0
2,3,"Alemanha, República Democrática",119512,238052,265742,429970,225086,393482,27715,138666,...,10794,45382,3660,25467,6261,32605,2698,6741,7630,45367
3,4,Angola,25931,49753,25721,71083,54786,84235,33557,189891,...,477,709,345,1065,0,0,0,0,4068,4761
4,5,Anguilla,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


🚨 Podemos notar que a coluna terminada em .1 se refere ao valor monetário (US$) e a coluna que apresenta somente o ano se refere a quantidade em (Kg). 

Deste modo podemos separar nossos dados em duas tabelas diferentes, uma para quantidade e outra para valor.

In [69]:
# Filtrar as colunas para quantidade exportada
exported_quantity = selected_export_data.melt(
    id_vars=["Id", "País"], var_name="Ano", value_name="exported_quantity"
)
exported_quantity = exported_quantity[
    exported_quantity["Ano"].apply(lambda x: not x.endswith(".1"))
]

# Filtrar as colunas para valor exportado
exported_value = selected_export_data.melt(
    id_vars=["Id", "País"], var_name="Ano", value_name="exported_value"
)
exported_value = exported_value[
    exported_value["Ano"].apply(lambda x: x.endswith(".1"))
]

# Remover o sufixo '.1' dos anos em valor exportado
exported_value["Ano"] = exported_value["Ano"].str.replace(".1", "")

In [70]:
exported_value.head()

,Id,País,Ano,exported_value
128,1,Afeganistão,2007,0
129,2,África do Sul,2007,0
130,3,"Alemanha, República Democrática",2007,238052
131,4,Angola,2007,49753
132,5,Anguilla,2007,0


In [71]:
exported_quantity.head()

,Id,País,Ano,exported_quantity
0,1,Afeganistão,2007,0
1,2,África do Sul,2007,0
2,3,"Alemanha, República Democrática",2007,119512
3,4,Angola,2007,25931
4,5,Anguilla,2007,0


In [72]:
merged_data = pd.merge(exported_quantity, exported_value, on=["Id", "País", "Ano"])
merged_data.rename(
    columns={"País": "País de destino", "exported_quantity": "Quantidade (kg/L)", "exported_value": "Valor em US$"},
    inplace=True,
)
merged_data["País de origem"] = "Brasil"
order = ["Id", "País de origem", "País de destino", "Ano", "Quantidade (kg/L)", "Valor em US$"]
merged_data = merged_data[order]
merged_data.head()

,Id,País de origem,País de destino,Ano,Quantidade (kg/L),Valor em US$
0,1,Brasil,Afeganistão,2007,0,0
1,2,Brasil,África do Sul,2007,0,0
2,3,Brasil,"Alemanha, República Democrática",2007,119512,238052
3,4,Brasil,Angola,2007,25931,49753
4,5,Brasil,Anguilla,2007,0,0


In [73]:
merged_data.to_csv("../data/exported_data2007_2022.csv", index=False)